# References


- https://github.com/MengtingWan/goodreads/blob/master/gdrive_id.csv (using goodreads_reviews_children)

- https://github.com/Reinalynn/Building-a-Book-Recommendation-System-using-Python/blob/master/Code/Text%20analysis%20-%20build%2C%20clean%2C%20prep%20review%20text.ipynb (using for merging review text with books metadata)

- https://medium.com/@armandj.olivares/building-nlp-content-based-recommender-systems-b104a709c042

- https://github.com/ArmandDS/jobs_recommendations/blob/master/job_analysis_content_recommendation.ipynb


# **Goal** 
To merge review text along with books metadata for goodreads_books_children dataset provided

**Inputs:**
- processed_data/books.csv
- processed_data/reviews.csv

**Outputs:**
- processed_data/books_with_review.csv


# Import libraries

In [ ]:
import pandas as pd

# Load data
Note: can be commented if run locally

In [ ]:
# from google.colab import drive

# drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# cd '/content/gdrive/MyDrive/CMPE_256-Prg_Ass1_014749488/'

/content/gdrive/MyDrive/CMPE_256-Prg_Ass1_014749488


# Read Books, Reviews data for merging

In [ ]:
book_df = pd.read_csv('processed_data/books.csv')

In [ ]:
book_df.head(2)

,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,similar_books,description,format,link,authors,publisher,num_pages,publication_day,isbn13,publication_month,edition_information,publication_year,url,image_url,book_id,ratings_count,work_id,title,title_without_series
0,1599150603,7,[],US,NaN,"[{'count': '56', 'name': 'to-read'}, {'count':...",NaN,False,4.13,B00DU10PUG,[],"Relates in vigorous prose the tale of Aeneas, ...",Paperback,https://www.goodreads.com/book/show/287141.The...,"[{'author_id': '3041852', 'role': ''}]",Yesterday's Classics,162.0,13.0,9781599150604,9.0,NaN,2006.0,https://www.goodreads.com/book/show/287141.The...,https://s.gr-assets.com/assets/nophoto/book/11...,287141,46,278578,The Aeneid for Boys and Girls,The Aeneid for Boys and Girls
1,1934876569,6,['151854'],US,NaN,"[{'count': '515', 'name': 'to-read'}, {'count'...",NaN,False,4.22,NaN,"['948696', '439885', '274955', '12978730', '37...","To Kara's astonishment, she discovers that a p...",Paperback,https://www.goodreads.com/book/show/6066812-al...,"[{'author_id': '19158', 'role': ''}]",Seven Seas,216.0,3.0,9781934876565,3.0,NaN,2009.0,https://www.goodreads.com/book/show/6066812-al...,https://images.gr-assets.com/books/1316637798m...,6066812,98,701117,All's Fairy in Love and War (Avalon: Web of Ma...,All's Fairy in Love and War (Avalon: Web of Ma...


In [ ]:
review_df =  pd.read_csv('processed_data/reviews.csv')

In [ ]:
review_df.head(2)

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,8842281e1d1347389f2ab93d60773d4d,17290220,22d424a2b0057b18fb6ecf017af7be92,5,One of my favorite books to read to my 5 year ...,Sat Nov 08 08:54:03 -0800 2014,Wed Jan 25 13:56:12 -0800 2017,Tue Jan 24 00:00:00 -0800 2017,NaN,4,0
1,8842281e1d1347389f2ab93d60773d4d,6954929,50ed4431c451d5677d98dd25ca8ec106,5,One of the best and most imaginative childrens...,Thu Oct 23 13:46:20 -0700 2014,Thu Oct 23 13:47:00 -0700 2014,NaN,NaN,6,1


In [ ]:
book_df['book_id'] = pd.to_numeric(book_df['book_id'])

In [ ]:
# add review_text from reviews2, combining by book_id
book_reviews_df = pd.merge(book_df, review_df, on = 'book_id', how = 'left')
book_reviews_df.head(2)

,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,similar_books,description,format,link,authors,publisher,num_pages,publication_day,isbn13,publication_month,edition_information,publication_year,url,image_url,book_id,ratings_count,work_id,title,title_without_series,user_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,1599150603,7,[],US,NaN,"[{'count': '56', 'name': 'to-read'}, {'count':...",NaN,False,4.13,B00DU10PUG,[],"Relates in vigorous prose the tale of Aeneas, ...",Paperback,https://www.goodreads.com/book/show/287141.The...,"[{'author_id': '3041852', 'role': ''}]",Yesterday's Classics,162.0,13.0,9781599150604,9.0,NaN,2006.0,https://www.goodreads.com/book/show/287141.The...,https://s.gr-assets.com/assets/nophoto/book/11...,287141,46,278578,The Aeneid for Boys and Girls,The Aeneid for Boys and Girls,fc0fba8c6f515fe897a455b16776a35a,aea34ea71d3542fef80c543dd9fe9fb2,4.0,"Once again, my kids loved this book about anci...",Mon May 09 19:19:00 -0700 2011,Mon Jun 13 17:07:25 -0700 2011,Wed Jun 01 00:00:00 -0700 2011,Mon May 09 00:00:00 -0700 2011,0.0,0.0
1,1599150603,7,[],US,NaN,"[{'count': '56', 'name': 'to-read'}, {'count':...",NaN,False,4.13,B00DU10PUG,[],"Relates in vigorous prose the tale of Aeneas, ...",Paperback,https://www.goodreads.com/book/show/287141.The...,"[{'author_id': '3041852', 'role': ''}]",Yesterday's Classics,162.0,13.0,9781599150604,9.0,NaN,2006.0,https://www.goodreads.com/book/show/287141.The...,https://s.gr-assets.com/assets/nophoto/book/11...,287141,46,278578,The Aeneid for Boys and Girls,The Aeneid for Boys and Girls,3b22aeaaa17316ec8d0de319dc2d6ca7,d6e7ff1dfb61513ff7a03ae898607235,5.0,Renders the story in straightforward prose whi...,Thu Dec 08 20:54:21 -0800 2016,Thu Dec 08 20:59:19 -0800 2016,Thu Dec 08 00:00:00 -0800 2016,Thu Dec 08 00:00:00 -0800 2016,0.0,0.0


In [ ]:
book_reviews_df = book_reviews_df[['book_id', 'title', 'review_text','ratings_count','num_pages','publisher','is_ebook','format','average_rating','publication_year', 'url', 'description', 'image_url', 'isbn']]
book_reviews_df.head()

,book_id,title,review_text,ratings_count,num_pages,publisher,is_ebook,format,average_rating,publication_year,url,description,image_url,isbn
0,287141,The Aeneid for Boys and Girls,"Once again, my kids loved this book about anci...",46,162.0,Yesterday's Classics,False,Paperback,4.13,2006.0,https://www.goodreads.com/book/show/287141.The...,"Relates in vigorous prose the tale of Aeneas, ...",https://s.gr-assets.com/assets/nophoto/book/11...,1599150603
1,287141,The Aeneid for Boys and Girls,Renders the story in straightforward prose whi...,46,162.0,Yesterday's Classics,False,Paperback,4.13,2006.0,https://www.goodreads.com/book/show/287141.The...,"Relates in vigorous prose the tale of Aeneas, ...",https://s.gr-assets.com/assets/nophoto/book/11...,1599150603
2,6066812,All's Fairy in Love and War (Avalon: Web of Ma...,"This was a really cute book, though to be hone...",98,216.0,Seven Seas,False,Paperback,4.22,2009.0,https://www.goodreads.com/book/show/6066812-al...,"To Kara's astonishment, she discovers that a p...",https://images.gr-assets.com/books/1316637798m...,1934876569
3,89378,Dog Heaven,"Really cute, sweet, and charming.",1331,40.0,Blue Sky Press,False,Hardcover,4.43,1995.0,https://www.goodreads.com/book/show/89378.Dog_...,In Newbery Medalist Cynthia Rylant's classic b...,https://images.gr-assets.com/books/1360057676m...,0590417010
4,89378,Dog Heaven,This beautiful children's book was anonymously...,1331,40.0,Blue Sky Press,False,Hardcover,4.43,1995.0,https://www.goodreads.com/book/show/89378.Dog_...,In Newbery Medalist Cynthia Rylant's classic b...,https://images.gr-assets.com/books/1360057676m...,0590417010


In [ ]:
book_reviews_df = book_reviews_df[['book_id', 'title', 'review_text','ratings_count','num_pages','publisher','is_ebook','format','average_rating','publication_year', 'url', 'description', 'image_url', 'isbn']].drop_duplicates()

In [ ]:
print(book_reviews_df.shape)

(734317, 14)


In [ ]:
book_reviews_df.head(2)

,book_id,title,review_text,ratings_count,num_pages,publisher,is_ebook,format,average_rating,publication_year,url,description,image_url,isbn
0,287141,The Aeneid for Boys and Girls,"Once again, my kids loved this book about anci...",46,162.0,Yesterday's Classics,False,Paperback,4.13,2006.0,https://www.goodreads.com/book/show/287141.The...,"Relates in vigorous prose the tale of Aeneas, ...",https://s.gr-assets.com/assets/nophoto/book/11...,1599150603
1,287141,The Aeneid for Boys and Girls,Renders the story in straightforward prose whi...,46,162.0,Yesterday's Classics,False,Paperback,4.13,2006.0,https://www.goodreads.com/book/show/287141.The...,"Relates in vigorous prose the tale of Aeneas, ...",https://s.gr-assets.com/assets/nophoto/book/11...,1599150603


## Clean review text

In [ ]:
import pandas as pd
import numpy as np
import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

from nltk.corpus import stopwords
import re
import string
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
stop = stopwords.words('english')
stop_words_ = set(stopwords.words('english'))
wn = WordNetLemmatizer()

import matplotlib.pyplot as plt

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
def black_txt(token):
    return token not in stop_words_ and token not in list(string.punctuation) and len(token)>2   
  
def clean_txt(text):
  clean_text = []
  clean_text2 = []
  text = re.sub("'", "",text)
  text=re.sub("(\\d|\\W)+"," ",text) 
  text = text.replace("nbsp", "")
  clean_text = [ wn.lemmatize(word, pos="v") for word in word_tokenize(text.lower()) if black_txt(word)]
  clean_text2 = [word for word in clean_text if black_txt(word)]
  return " ".join(clean_text2)

In [ ]:
book_reviews_df['review_text'] = book_reviews_df['review_text'].map(str).apply(clean_txt)
book_reviews_df.head(2)

,book_id,title,review_text,ratings_count,num_pages,publisher,is_ebook,format,average_rating,publication_year,url,description,image_url,isbn
0,287141,The Aeneid for Boys and Girls,kid love book ancient history,46,162.0,Yesterday's Classics,False,Paperback,4.13,2006.0,https://www.goodreads.com/book/show/287141.The...,"Relates in vigorous prose the tale of Aeneas, ...",https://s.gr-assets.com/assets/nophoto/book/11...,1599150603
1,287141,The Aeneid for Boys and Girls,render story straightforward prose retain much...,46,162.0,Yesterday's Classics,False,Paperback,4.13,2006.0,https://www.goodreads.com/book/show/287141.The...,"Relates in vigorous prose the tale of Aeneas, ...",https://s.gr-assets.com/assets/nophoto/book/11...,1599150603


In [ ]:
book_reviews_df.isna().sum()

book_id                 0
title                   2
review_text             0
ratings_count           0
num_pages           94334
publisher           84914
is_ebook                0
format              97501
average_rating          0
publication_year    82213
url                     0
description         43570
image_url               0
isbn                54583
dtype: int64

In [ ]:
books_with_reviews_df = book_reviews_df[['book_id', 'title', 'review_text','ratings_count','is_ebook','average_rating','publication_year', 'url', 'description', 'image_url', 'isbn']]

In [ ]:
print(books_with_reviews_df.shape)

(734317, 11)


# Save merged book data (with reviews)

In [ ]:
books_with_reviews_df.to_csv('processed_data/books_with_review.csv')
books_with_reviews_df.head(2)

,book_id,title,review_text,ratings_count,is_ebook,average_rating,publication_year,url,description,image_url,isbn
0,287141,The Aeneid for Boys and Girls,kid love book ancient history,46,False,4.13,2006.0,https://www.goodreads.com/book/show/287141.The...,"Relates in vigorous prose the tale of Aeneas, ...",https://s.gr-assets.com/assets/nophoto/book/11...,1599150603
1,287141,The Aeneid for Boys and Girls,render story straightforward prose retain much...,46,False,4.13,2006.0,https://www.goodreads.com/book/show/287141.The...,"Relates in vigorous prose the tale of Aeneas, ...",https://s.gr-assets.com/assets/nophoto/book/11...,1599150603
